In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
#Load in cbb dataset
cbb = pd.read_csv('cbb.csv')
cbb.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [3]:
#Now lets look at the span of years thhis covers
cbb['YEAR'].min(), cbb['YEAR'].max()

(2013, 2024)

Okay great, this data covers from 2013-2024. Now while data before 2013 exists, there was a paradigm shift in the cbb landscape around that time in terms of data tracking and analytics, so we will not search for any data prior to this point (multiple discussions on the use of the 3pt line, foul rules etc.).

The data for 2025 has concluded and it is not included here (this is obvious, since Florida was the national champs in 2025 and is not present here). We need to add in the cbb25.csv data to the end of this. NOTE: It may seem convenient to just use the 2025 data as the test/validation data for this project, but there has recently been another paradigm shift in CBB (college sports in general since 2023/2024), with the addition of the transfer portal and NIL. Even analysts within professional sports and prediction platforms have not found a good way to account for this, so I will add it into this dataset. We can explore this though to see differences year to year. 

In [4]:
# Load in cbb25 dataset and compare headers to cbb
cbb25 = pd.read_csv('cbb25.csv')
print("CBB Columns:", cbb.columns,'\n', "CBB25 Columns:", cbb25.columns)
print("Columns in cbb but not in cbb25:", cbb.columns.difference(cbb25.columns))
print("Columns in cbb25 but not in cbb:", cbb25.columns.difference(cbb.columns))


CBB Columns: Index(['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'POSTSEASON', 'SEED', 'YEAR'],
      dtype='object') 
 CBB25 Columns: Index(['RK', 'Team', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O',
       'EFG_D', 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D',
       '3P_O', '3P_D', '3PR', '3PRD', 'ADJ_T', 'WAB', 'SEED'],
      dtype='object')
Columns in cbb but not in cbb25: Index(['POSTSEASON', 'TEAM', 'YEAR'], dtype='object')
Columns in cbb25 but not in cbb: Index(['3PR', '3PRD', 'RK', 'Team'], dtype='object')


In [5]:
#change 'Team' to 'TEAM' in cbb25 to match cbb
cbb25 = cbb25.rename(columns={'Team':'TEAM'})
#drop the postseason column from cbb to match cbb25
cbb = cbb.drop(columns=['POSTSEASON'])
#Add year 2025 to cbb25
cbb25['YEAR'] = 2025

#Remove the rk columns from cbb25 to match cbb
cbb25 = cbb25.drop(columns=['RK'])

#The combined dataset does nto include 3PR (3-point rate) or 3PRD (3-point rate defense), so we will drop these columns from cbb25
cbb25 = cbb25.drop(columns=['3PR', '3PRD'])

#These two stats are probably very important in reality, but to start we will not consider them as they may not exist for every season in teh individual datasets. This stat is also much more nuanced, as being a worse team may lead to a higher rate of 3pt attemps per possession, but the best teams may also shoot more 3pts as well.


In [6]:
#now check again
print("CBB Columns after cleaning:", cbb.columns,'\n', "CBB25 Columns after cleaning:", cbb25.columns)
print("Columns in cbb but not in cbb25 after cleaning:", cbb.columns.difference(cbb25.columns))
print("Columns in cbb25 but not in cbb after cleaning:", cbb25.columns.difference(cbb.columns))

CBB Columns after cleaning: Index(['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'YEAR'],
      dtype='object') 
 CBB25 Columns after cleaning: Index(['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'YEAR'],
      dtype='object')
Columns in cbb but not in cbb25 after cleaning: Index([], dtype='object')
Columns in cbb25 but not in cbb after cleaning: Index([], dtype='object')


In [7]:
#Now we can combine the datasets
cbb_combined = pd.concat([cbb, cbb25], ignore_index=True)
#Now lets look at the span of years thhis covers
cbb_combined['YEAR'].min(), cbb_combined['YEAR'].max()

(2013, 2025)

In [8]:
#list of columns in the combined dataset
cbb_combined.columns.tolist()

['TEAM',
 'CONF',
 'G',
 'W',
 'ADJOE',
 'ADJDE',
 'BARTHAG',
 'EFG_O',
 'EFG_D',
 'TOR',
 'TORD',
 'ORB',
 'DRB',
 'FTR',
 'FTRD',
 '2P_O',
 '2P_D',
 '3P_O',
 '3P_D',
 'ADJ_T',
 'WAB',
 'SEED',
 'YEAR']

In [9]:
#Add win percentage column for modeling
cbb_combined['WIN_PCT'] = cbb_combined['W'] / cbb_combined['G']

#We dont need wins/losses or games for modeling, so we can drop these columns
cbb_combined = cbb_combined.drop(columns=['W', 'G'])

#I will leave the seed in for now, but we may want to drop it later as well


In [10]:

#I will drop the WAB column as well, as it is redundant with other stats and is hard to interpret
cbb_combined = cbb_combined.drop(columns=['WAB'])

Great, now these are combined and I can save this new dataset as its own csv for future use. 

In [11]:
cbb_combined.to_csv('cbb_full_v2.csv', index=False) #this dataset is good for data exploration